##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Tutorial de componentes TFX Keras

***Una introducción detallada de los componentes de TensorFlow Extended (TFX)***

Nota: Recomendamos ejecutar este tutorial en un bloc de notas de Colab, ¡no es necesario configurarlo! Simplemente haga clic en "Ejecutar en Google Colab".

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/components_keras"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/tfx/components_keras.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/tfx/components_keras.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a></td>
<td><a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tfx/tutorials/tfx/components_keras.ipynb"><img width="32px" src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table></div>

En este tutorial basado en Colab repasaremos de forma interactiva cada componente integrado de TensorFlow Extended (TFX).

Cubre cada paso de un proceso de aprendizaje automático de un extremo a otro, desde la ingesta de datos hasta la inserción de un modelo en servicio.

Cuando haya terminado, el contenido de este bloc de notas se puede exportar automáticamente como código fuente de canalización de TFX, que podrá orquestar con Apache Airflow y Apache Beam.

Nota: Este bloc de notas demuestra el uso de modelos nativos de Keras en canalizaciones de TFX. **TFX solo admite la versión TensorFlow 2 de Keras**.

## Antecedentes

En este bloc de notas se demuestra cómo utilizar TFX en un entorno Jupyter/Colab. Aquí, analizamos el ejemplo Taxi de Chicago en un bloc de notas interactivo.

Trabajar en un bloc de notas interactivo es una forma útil de familiarizarse con la estructura de una canalización de TFX. También es útil a la hora de desarrollar sus propias canalizaciones como un entorno de desarrollo ligero, pero debe tener en cuenta que existen diferencias en la forma en que se organizan los bloc de notas interactivos y en cómo acceden a los artefactos de metadatos.

### Orquestación

En una implementación de producción de TFX, se usará un orquestador como Apache Airflow, Kubeflow Pipelines o Apache Beam para orquestar un grafo de canalización predefinido de componentes de TFX. En un bloc de notas interactivo, el bloc de notas en sí es el orquestador y ejecuta cada componente de TFX a medida que se ejecutan las celdas del bloc de notas.

### Metadatos

En una implementación de producción de TFX, accederá a los metadatos a través de la API ML Metadata (MLMD). MLMD almacena propiedades de metadatos en una base de datos como MySQL o SQLite, y almacena las cargas útiles de metadatos en un almacén persistente, como en su sistema de archivos. En un bloc de notas interactivo, tanto las propiedades como las cargas útiles se almacenan en una base de datos SQLite efímera en el directorio `/tmp` del bloc de notas Jupyter o del servidor Colab.

## Preparación

En primer lugar, tenemos que instalar e importar los paquetes necesarios, configurar rutas y descargar datos.

### Actualización de pip

Para evitar actualizar Pip en un sistema cuando se ejecuta localmente, verifique que se esté ejecutando en Colab. Por supuesto, los sistemas locales se pueden actualizar por separado.

In [ ]:
import sys
if 'google.colab' in sys.modules:
  !pip install --upgrade pip

### Instalación de TFX

**Nota: En Google Colab, debido a las actualizaciones de paquetes, la primera vez que ejecuta esta celda debe reiniciar el tiempo de ejecución (Tiempo de ejecución &gt; Reiniciar tiempo de ejecución...).**

In [ ]:
!pip install tfx

### Desinstalación de shapely

TODO(b/263441833) Esta es una solución temporal para evitar un ImportError. En última instancia, debería solucionarse admitiendo una versión reciente de Bigquery, en lugar de desinstalar otras dependencias adicionales.

In [ ]:
!pip uninstall shapely -y

## ¿Reinició el tiempo de ejecución?

Si está usando Google Colab, la primera vez que ejecute la celda anterior, debe reiniciar el tiempo de ejecución (Tiempo de ejecución &gt; Reiniciar tiempo de ejecución...). Esto se debe a la forma en que Colab carga los paquetes.

### Importación de paquetes

Importamos los paquetes necesarios, incluidas las clases de componentes de TFX estándar.

In [ ]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

Revisemos las versiones de la biblioteca.

In [ ]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

### Configuración de rutas de canalización

In [ ]:
# This is the root directory for your TFX pip package installation.
_tfx_root = tfx.__path__[0]

# This is the directory containing the TFX Chicago Taxi Pipeline example.
_taxi_root = os.path.join(_tfx_root, 'examples/chicago_taxi_pipeline')

# This is the path where your model will be pushed for serving.
_serving_model_dir = os.path.join(
    tempfile.mkdtemp(), 'serving_model/taxi_simple')

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

### Descarga de datos de muestra

Descargamos el conjunto de datos de ejemplo para usarlo en nuestra canalización de TFX.

El conjunto de datos que estamos usando es el [conjunto de datos Taxi Trips](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew) publicado por la ciudad de Chicago. Las columnas de este conjunto de datos son las siguientes:

<table>
<tr>
<td>pickup_community_area</td>
<td>fare</td>
<td>trip_start_month</td>
</tr>
<tr>
<td>trip_start_hour</td>
<td>trip_start_day</td>
<td>trip_start_timestamp</td>
</tr>
<tr>
<td>pickup_latitude</td>
<td>pickup_longitude</td>
<td>dropoff_latitude</td>
</tr>
<tr>
<td>dropoff_longitude</td>
<td>trip_miles</td>
<td>pickup_census_tract</td>
</tr>
<tr>
<td>dropoff_census_tract</td>
<td>payment_type</td>
<td>company</td>
</tr>
<tr>
<td>trip_seconds</td>
<td>dropoff_community_area</td>
<td>tips</td>
</tr>
</table>

Con este conjunto de datos, compilaremos un modelo que predice `tips` de un viaje.

In [ ]:
_data_root = tempfile.mkdtemp(prefix='tfx-data')
DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv'
_data_filepath = os.path.join(_data_root, "data.csv")
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

Veamos rápidamente al archivo CSV.

In [ ]:
!head {_data_filepath}

*Descargo de responsabilidad: Este sitio ofrece aplicaciones que usan datos que fueron modificados para su uso desde su fuente original, www.cityofchicago.org, el sitio web oficial de la ciudad de Chicago. La ciudad de Chicago no garantiza el contenido, la exactitud, la puntualidad o la integridad de ninguno de los datos que se proporcionan en este sitio. Los datos proporcionados en este sitio están sujetos a cambios en cualquier momento. Se entiende que los datos proporcionados en este sitio se usan bajo su propia responsabilidad.*

### Cómo crear el InteractiveContext

Por último, creamos un InteractiveContext, que nos permitirá ejecutar componentes de TFX de forma interactiva en este bloc de notas.

In [ ]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
context = InteractiveContext()

## Cómo ejecutar componentes de TFX de forma interactiva

En las siguientes celdas, creamos componentes de TFX uno por uno, ejecutamos cada uno de ellos y visualizamos sus artefactos de salida.

### ExampleGen

El componente `ExampleGen` suele estar al inicio de una canalización de TFX. Será el encargado de lo siguiente:

1. Dividir los datos en conjuntos de entrenamiento y evaluación (de forma predeterminada, 2/3 de entrenamiento + 1/3 de evaluación)
2. Convertir datos al formato `tf.Example` (obtenga más información [aquí](https://www.tensorflow.org/tutorials/load_data/tfrecord))
3. Copiar los datos en el directorio `_tfx_root` para que otros componentes puedan acceder

`ExampleGen` toma como entrada la ruta a su fuente de datos. En nuestro caso, esta es la ruta `_data_root` que contiene el CSV descargado.

Nota: En este bloc de notas, podemos crear instancias de componentes uno por uno y ejecutarlos con `InteractiveContext.run()`. Por el contrario, en un entorno de producción, especificaríamos todos los componentes por adelantado en una `Pipeline` para pasarlos al orquestador (consulte la [Guía para compilar una canalización de TFX](https://www.tensorflow.org/tfx/guide/build_tfx_pipeline)).

#### Cómo habilitar el caché

Al utilizar `InteractiveContext` en un bloc de notas para desarrollar una canalización, puede controlar en qué momentos los componentes individuales almacenarán sus resultados en caché. Establezca `enable_cache` en `True` cuando desee reutilizar los artefactos de salida anteriores que generó el componente. Establezca `enable_cache` en `False` cuando desee volver a calcular los artefactos de salida de un componente, si está realizando cambios en el código, por ejemplo.

In [ ]:
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)
context.run(example_gen, enable_cache=True)

Examinemos los artefactos de salida de `ExampleGen`. Este componente produce dos artefactos, ejemplos de entrenamiento y ejemplos de evaluación:

In [ ]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

También podemos echar un vistazo a los tres primeros ejemplos de entrenamiento:

In [ ]:
# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

Ahora que `ExampleGen` ha terminado de ingerir los datos, el siguiente paso es el análisis de datos.

### StatisticsGen

El componente `StatisticsGen` calcula estadísticas sobre su conjunto de datos para el análisis de datos, así como para su uso en componentes posteriores. Usa la biblioteca [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started).

`StatisticsGen` toma como entrada el conjunto de datos que acabamos de ingerir con ayuda de `ExampleGen`.

In [ ]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen, enable_cache=True)

Una vez que `StatisticsGen` termina de ejecutarse, podemos ver las estadísticas generadas. Intente experimentar con los diferentes gráficos.

In [ ]:
context.show(statistics_gen.outputs['statistics'])

### SchemaGen

El componente `SchemaGen` genera un esquema basado en sus estadísticas de datos. (Un esquema define los límites, tipos y propiedades esperados de las características de su conjunto de datos). También usa la biblioteca [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started).

Nota: El esquema generado es el de mejor esfuerzo y solo intenta inferir propiedades básicas de los datos. Se espera que lo revise y modifique según sea necesario.

`SchemaGen` tomará como entrada las estadísticas que generamos con `StatisticsGen`, tomando como referencia la división del entrenamiento de forma predeterminada.

In [ ]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen, enable_cache=True)

Una vez que `SchemaGen` termine de ejecutarse, podremos visualizar el esquema generado como una tabla.

In [ ]:
context.show(schema_gen.outputs['schema'])

Cada característica del conjunto de datos aparece como una fila en la tabla del esquema, junto con sus propiedades. El esquema también captura todos los valores que toma una característica categórica, denominada dominio.

Para obtener más información sobre los esquemas, consulte [la documentación de SchemaGen](https://www.tensorflow.org/tfx/guide/schemagen).

### ExampleValidator

El componente `ExampleValidator` detecta anomalías en sus datos, según las expectativas definidas por el esquema. También utiliza la biblioteca [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started).

`ExampleValidator` tomará como entrada las estadísticas de `StatisticsGen` y el esquema de `SchemaGen`.

In [ ]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator, enable_cache=True)

Una vez que `ExampleValidator` termine de ejecutarse, podremos visualizar las anomalías como una tabla.

In [ ]:
context.show(example_validator.outputs['anomalies'])

En la tabla de anomalías podemos ver que no hay anomalías. Esto es lo que esperaríamos, ya que este es el primer conjunto de datos que analizamos y el esquema se adapta a él. Debe revisar este esquema: cualquier cosa inesperada significa una anomalía en los datos. Una vez revisado, el esquema se puede usar para proteger datos futuros y las anomalías producidas aquí se pueden usar para depurar el rendimiento del modelo, comprender cómo evolucionan los datos con el tiempo e identificar errores en los datos.

### Transform

El componente `Transform` ejecuta ingeniería de características tanto para el entrenamiento como para el servicio. Utiliza la biblioteca [TensorFlow Transform](https://www.tensorflow.org/tfx/transform/get_started).

`Transform` tomará como entrada los datos de `ExampleGen`, el esquema de `SchemaGen`, así como un módulo que contiene código Transform definido por el usuario.

A continuación, veamos un ejemplo de código de Transform definido por el usuario a continuación (para obtener una introducción a las API de TensorFlow Transform, [consulte el tutorial](https://www.tensorflow.org/tfx/tutorials/transform/simple)). Primero, definimos algunas constantes para la ingeniería de características:

Nota: La magia de celda `%%writefile` guardará el contenido de la celda como un archivo `.py` en el disco. Esto permite que el componente `Transform` cargue su código como un módulo.


In [ ]:
_taxi_constants_module_file = 'taxi_constants.py'

In [ ]:
%%writefile {_taxi_constants_module_file}

NUMERICAL_FEATURES = ['trip_miles', 'fare', 'trip_seconds']

BUCKET_FEATURES = [
    'pickup_latitude', 'pickup_longitude', 'dropoff_latitude',
    'dropoff_longitude'
]
# Number of buckets used by tf.transform for encoding each feature.
FEATURE_BUCKET_COUNT = 10

CATEGORICAL_NUMERICAL_FEATURES = [
    'trip_start_hour', 'trip_start_day', 'trip_start_month',
    'pickup_census_tract', 'dropoff_census_tract', 'pickup_community_area',
    'dropoff_community_area'
]

CATEGORICAL_STRING_FEATURES = [
    'payment_type',
    'company',
]

# Number of vocabulary terms used for encoding categorical features.
VOCAB_SIZE = 1000

# Count of out-of-vocab buckets in which unrecognized categorical are hashed.
OOV_SIZE = 10

# Keys
LABEL_KEY = 'tips'
FARE_KEY = 'fare'

def t_name(key):
  """
  Rename the feature keys so that they don't clash with the raw keys when
  running the Evaluator component.
  Args:
    key: The original feature key
  Returns:
    key with '_xf' appended
  """
  return key + '_xf'

A continuación, escribimos un `preprocessing_fn` que toma datos sin procesar como entrada y devuelve características transformadas en las que nuestro modelo puede entrenar:

In [ ]:
_taxi_transform_module_file = 'taxi_transform.py'

In [ ]:
%%writefile {_taxi_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

# Imported files such as taxi_constants are normally cached, so changes are
# not honored after the first import.  Normally this is good for efficiency, but
# during development when we may be iterating code it can be a problem. To
# avoid this problem during development, reload the file.
import taxi_constants
import sys
if 'google.colab' in sys.modules:  # Testing to see if we're doing development
  import importlib
  importlib.reload(taxi_constants)

_NUMERICAL_FEATURES = taxi_constants.NUMERICAL_FEATURES
_BUCKET_FEATURES = taxi_constants.BUCKET_FEATURES
_FEATURE_BUCKET_COUNT = taxi_constants.FEATURE_BUCKET_COUNT
_CATEGORICAL_NUMERICAL_FEATURES = taxi_constants.CATEGORICAL_NUMERICAL_FEATURES
_CATEGORICAL_STRING_FEATURES = taxi_constants.CATEGORICAL_STRING_FEATURES
_VOCAB_SIZE = taxi_constants.VOCAB_SIZE
_OOV_SIZE = taxi_constants.OOV_SIZE
_FARE_KEY = taxi_constants.FARE_KEY
_LABEL_KEY = taxi_constants.LABEL_KEY


def _make_one_hot(x, key):
  """Make a one-hot tensor to encode categorical features.
  Args:
    X: A dense tensor
    key: A string key for the feature in the input
  Returns:
    A dense one-hot tensor as a float list
  """
  integerized = tft.compute_and_apply_vocabulary(x,
          top_k=_VOCAB_SIZE,
          num_oov_buckets=_OOV_SIZE,
          vocab_filename=key, name=key)
  depth = (
      tft.experimental.get_vocabulary_size_by_name(key) + _OOV_SIZE)
  one_hot_encoded = tf.one_hot(
      integerized,
      depth=tf.cast(depth, tf.int32),
      on_value=1.0,
      off_value=0.0)
  return tf.reshape(one_hot_encoded, [-1, depth])


def _fill_in_missing(x):
  """Replace missing values in a SparseTensor.
  Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
  Args:
    x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
      in the second dimension.
  Returns:
    A rank 1 tensor where missing values of `x` have been filled in.
  """
  if not isinstance(x, tf.sparse.SparseTensor):
    return x

  default_value = '' if x.dtype == tf.string else 0
  return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)


def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.
  Args:
    inputs: map from feature keys to raw not-yet-transformed features.
  Returns:
    Map from string feature key to transformed feature operations.
  """
  outputs = {}
  for key in _NUMERICAL_FEATURES:
    # If sparse make it dense, setting nan's to 0 or '', and apply zscore.
    outputs[taxi_constants.t_name(key)] = tft.scale_to_z_score(
        _fill_in_missing(inputs[key]), name=key)

  for key in _BUCKET_FEATURES:
    outputs[taxi_constants.t_name(key)] = tf.cast(tft.bucketize(
            _fill_in_missing(inputs[key]), _FEATURE_BUCKET_COUNT, name=key),
            dtype=tf.float32)

  for key in _CATEGORICAL_STRING_FEATURES:
    outputs[taxi_constants.t_name(key)] = _make_one_hot(_fill_in_missing(inputs[key]), key)

  for key in _CATEGORICAL_NUMERICAL_FEATURES:
    outputs[taxi_constants.t_name(key)] = _make_one_hot(tf.strings.strip(
        tf.strings.as_string(_fill_in_missing(inputs[key]))), key)

  # Was this passenger a big tipper?
  taxi_fare = _fill_in_missing(inputs[_FARE_KEY])
  tips = _fill_in_missing(inputs[_LABEL_KEY])
  outputs[_LABEL_KEY] = tf.where(
      tf.math.is_nan(taxi_fare),
      tf.cast(tf.zeros_like(taxi_fare), tf.int64),
      # Test if the tip was > 20% of the fare.
      tf.cast(
          tf.greater(tips, tf.multiply(taxi_fare, tf.constant(0.2))), tf.int64))

  return outputs

Ahora, pasamos este código de ingeniería de características al componente `Transform` y lo ejecutamos para transformar sus datos.

In [ ]:
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_taxi_transform_module_file))
context.run(transform, enable_cache=True)

Examinemos los artefactos de salida de `Transform`. Este componente produce dos tipos de resultados:

- `transform_graph` es el grafo que puede realizar las operaciones de preprocesamiento (este grafo se incluirá en los modelos de servicio y evaluación).
- `transformed_examples` representa los datos de evaluación y entrenamiento preprocesados.

In [ ]:
transform.outputs

Echemos un vistazo al artefacto `transform_graph`. Apunta a un directorio que contiene tres subdirectorios.

In [ ]:
train_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(train_uri)

El subdirectorio `transformed_metadata` contiene el esquema de los datos preprocesados. El subdirectorio `transform_fn` contiene el grafo de preprocesamiento real. El subdirectorio `metadata` contiene el esquema de los datos originales.

También podemos echar un vistazo a los primeros tres ejemplos transformados:

In [ ]:
# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

Después de que el componente `Transform` haya transformado sus datos en características, el siguiente paso es entrenar un modelo.

### Trainer

El componente `Trainer` entrenará un modelo que usted defina en TensorFlow. Trainer admite de forma predeterminada la API Estimator. Para usar la API de Keras, debe especificar [Trainer genérico](https://github.com/tensorflow/community/blob/master/rfcs/20200117-tfx-generic-trainer.md) al configurar `custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor)` en el constructor de Trainer.

`Trainer` toma como entrada el esquema de `SchemaGen`, los datos transformados y el grafo de `Transform`, los parámetros de entrenamiento, así como un módulo que contiene código de modelo definido por el usuario.

Veamos un ejemplo de código de modelo definido por el usuario a continuación (para obtener una introducción a las API de TensorFlow Keras, [consulte el tutorial](https://www.tensorflow.org/guide/keras)):

In [ ]:
_taxi_trainer_module_file = 'taxi_trainer.py'

In [ ]:
%%writefile {_taxi_trainer_module_file}

from typing import Dict, List, Text

import os
import glob
from absl import logging

import datetime
import tensorflow as tf
import tensorflow_transform as tft

from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tensorflow_transform import TFTransformOutput

# Imported files such as taxi_constants are normally cached, so changes are
# not honored after the first import.  Normally this is good for efficiency, but
# during development when we may be iterating code it can be a problem. To
# avoid this problem during development, reload the file.
import taxi_constants
import sys
if 'google.colab' in sys.modules:  # Testing to see if we're doing development
  import importlib
  importlib.reload(taxi_constants)

_LABEL_KEY = taxi_constants.LABEL_KEY

_BATCH_SIZE = 40


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      tf_transform_output.transformed_metadata.schema)

def _get_tf_examples_serving_signature(model, tf_transform_output):
  """Returns a serving signature that accepts `tensorflow.Example`."""

  # We need to track the layers in the model in order to save it.
  # TODO(b/162357359): Revise once the bug is resolved.
  model.tft_layer_inference = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def serve_tf_examples_fn(serialized_tf_example):
    """Returns the output to be used in the serving signature."""
    raw_feature_spec = tf_transform_output.raw_feature_spec()
    # Remove label feature since these will not be present at serving time.
    raw_feature_spec.pop(_LABEL_KEY)
    raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
    transformed_features = model.tft_layer_inference(raw_features)
    logging.info('serve_transformed_features = %s', transformed_features)

    outputs = model(transformed_features)
    # TODO(b/154085620): Convert the predicted labels from the model using a
    # reverse-lookup (opposite of transform.py).
    return {'outputs': outputs}

  return serve_tf_examples_fn


def _get_transform_features_signature(model, tf_transform_output):
  """Returns a serving signature that applies tf.Transform to features."""

  # We need to track the layers in the model in order to save it.
  # TODO(b/162357359): Revise once the bug is resolved.
  model.tft_layer_eval = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def transform_features_fn(serialized_tf_example):
    """Returns the transformed_features to be fed as input to evaluator."""
    raw_feature_spec = tf_transform_output.raw_feature_spec()
    raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
    transformed_features = model.tft_layer_eval(raw_features)
    logging.info('eval_transformed_features = %s', transformed_features)
    return transformed_features

  return transform_features_fn


def export_serving_model(tf_transform_output, model, output_dir):
  """Exports a keras model for serving.
  Args:
    tf_transform_output: Wrapper around output of tf.Transform.
    model: A keras model to export for serving.
    output_dir: A directory where the model will be exported to.
  """
  # The layer has to be saved to the model for keras tracking purpases.
  model.tft_layer = tf_transform_output.transform_features_layer()

  signatures = {
      'serving_default':
          _get_tf_examples_serving_signature(model, tf_transform_output),
      'transform_features':
          _get_transform_features_signature(model, tf_transform_output),
  }

  model.save(output_dir, save_format='tf', signatures=signatures)


def _build_keras_model(tf_transform_output: TFTransformOutput
                       ) -> tf.keras.Model:
  """Creates a DNN Keras model for classifying taxi data.

  Args:
    tf_transform_output: [TFTransformOutput], the outputs from Transform

  Returns:
    A keras Model.
  """
  feature_spec = tf_transform_output.transformed_feature_spec().copy()
  feature_spec.pop(_LABEL_KEY)

  inputs = {}
  for key, spec in feature_spec.items():
    if isinstance(spec, tf.io.VarLenFeature):
      inputs[key] = tf.keras.layers.Input(
          shape=[None], name=key, dtype=spec.dtype, sparse=True)
    elif isinstance(spec, tf.io.FixedLenFeature):
      # TODO(b/208879020): Move into schema such that spec.shape is [1] and not
      # [] for scalars.
      inputs[key] = tf.keras.layers.Input(
          shape=spec.shape or [1], name=key, dtype=spec.dtype)
    else:
      raise ValueError('Spec type is not supported: ', key, spec)
  
  output = tf.keras.layers.Concatenate()(tf.nest.flatten(inputs))
  output = tf.keras.layers.Dense(100, activation='relu')(output)
  output = tf.keras.layers.Dense(70, activation='relu')(output)
  output = tf.keras.layers.Dense(50, activation='relu')(output)
  output = tf.keras.layers.Dense(20, activation='relu')(output)
  output = tf.keras.layers.Dense(1)(output)
  return tf.keras.Model(inputs=inputs, outputs=output)


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

  train_dataset = _input_fn(fn_args.train_files, fn_args.data_accessor, 
                            tf_transform_output, _BATCH_SIZE)
  eval_dataset = _input_fn(fn_args.eval_files, fn_args.data_accessor, 
                           tf_transform_output, _BATCH_SIZE)

  model = _build_keras_model(tf_transform_output)

  model.compile(
      loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
      optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
      metrics=[tf.keras.metrics.BinaryAccuracy()])

  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=fn_args.model_run_dir, update_freq='batch')

  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps,
      callbacks=[tensorboard_callback])

  # Export the model.
  export_serving_model(tf_transform_output, model, fn_args.serving_model_dir)

Ahora, pasamos este código de modelo al componente `Trainer` y lo ejecutamos para entrenar el modelo.

In [ ]:
trainer = tfx.components.Trainer(
    module_file=os.path.abspath(_taxi_trainer_module_file),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=tfx.proto.TrainArgs(num_steps=10000),
    eval_args=tfx.proto.EvalArgs(num_steps=5000))
context.run(trainer, enable_cache=True)

#### Análisis del entrenamiento con TensorBoard

Echemos un vistazo al artefacto Trainer. Apunta a un directorio que contiene los subdirectorios del modelo.

In [ ]:
model_artifact_dir = trainer.outputs['model'].get()[0].uri
pp.pprint(os.listdir(model_artifact_dir))
model_dir = os.path.join(model_artifact_dir, 'Format-Serving')
pp.pprint(os.listdir(model_dir))

Opcionalmente, podemos conectar TensorBoard al Trainer para analizar las curvas de entrenamiento de nuestro modelo.

In [ ]:
model_run_artifact_dir = trainer.outputs['model_run'].get()[0].uri

%load_ext tensorboard
%tensorboard --logdir {model_run_artifact_dir}

### Evaluator

El componente `Evaluator` calcula las métricas de rendimiento del modelo sobre el conjunto de evaluación. Utiliza la biblioteca [TensorFlow Data Analysis](https://www.tensorflow.org/tfx/model_analysis/get_started). Opcionalmente, `Evaluator` también puede validar que un modelo recién entrenado es mejor que el modelo anterior. Esto es útil en una configuración de canalización de producción donde se puede entrenar y validar automáticamente un modelo todos los días. En este bloc de notas, solo entrenamos un modelo, por lo que `Evaluator` automáticamente etiquetará el modelo como "bueno".

`Evaluator` tomará como entrada los datos de `ExampleGen`, el modelo entrenado de `Trainer` y la configuración de segmentación. La configuración de segmentación le permite segmentar sus métricas según los valores de las características (por ejemplo, ¿cómo se desempeña su modelo en viajes en taxi que comienzan a las 8 a. m. versus a las 8 p. m.?). Vea un ejemplo de esta configuración a continuación:

In [ ]:
# Imported files such as taxi_constants are normally cached, so changes are
# not honored after the first import.  Normally this is good for efficiency, but
# during development when we may be iterating code it can be a problem. To
# avoid this problem during development, reload the file.
import taxi_constants
import sys
if 'google.colab' in sys.modules:  # Testing to see if we're doing development
  import importlib
  importlib.reload(taxi_constants)

eval_config = tfma.EvalConfig(
    model_specs=[
        # This assumes a serving model with signature 'serving_default'. If
        # using estimator based EvalSavedModel, add signature_name: 'eval' and
        # remove the label_key.
        tfma.ModelSpec(
            signature_name='serving_default',
            label_key=taxi_constants.LABEL_KEY,
            preprocessing_function_names=['transform_features'],
            )
        ],
    metrics_specs=[
        tfma.MetricsSpec(
            # The metrics added here are in addition to those saved with the
            # model (assuming either a keras model or EvalSavedModel is used).
            # Any metrics added into the saved model (for example using
            # model.compile(..., metrics=[...]), etc) will be computed
            # automatically.
            # To add validation thresholds for metrics saved with the model,
            # add them keyed by metric name to the thresholds map.
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='BinaryAccuracy',
                  threshold=tfma.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          lower_bound={'value': 0.5}),
                      # Change threshold will be ignored if there is no
                      # baseline model resolved from MLMD (first run).
                      change_threshold=tfma.GenericChangeThreshold(
                          direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                          absolute={'value': -1e-10})))
            ]
        )
    ],
    slicing_specs=[
        # An empty slice spec means the overall slice, i.e. the whole dataset.
        tfma.SlicingSpec(),
        # Data can be sliced along a feature column. In this case, data is
        # sliced along feature column trip_start_hour.
        tfma.SlicingSpec(
            feature_keys=['trip_start_hour'])
    ])

A continuación, le pasamos esta configuración `Evaluator` y lo ejecutamos.

In [ ]:
# Use TFMA to compute a evaluation statistics over features of a model and
# validate them against a baseline.

# The model resolver is only required if performing model validation in addition
# to evaluation. In this case we validate against the latest blessed model. If
# no model has been blessed before (as in this case) the evaluator will make our
# candidate the first blessed model.
model_resolver = tfx.dsl.Resolver(
      strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.dsl.Channel(
          type=tfx.types.standard_artifacts.ModelBlessing)).with_id(
              'latest_blessed_model_resolver')
context.run(model_resolver, enable_cache=True)

evaluator = tfx.components.Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
context.run(evaluator, enable_cache=True)

Ahora examinemos los artefactos de salida de `Evaluator`.

In [ ]:
evaluator.outputs

Al usar el resultado `evaluation` podemos mostrar la visualización predeterminada de métricas globales en todo el conjunto de evaluación.

In [ ]:
context.show(evaluator.outputs['evaluation'])

Para ver la visualización de métricas de evaluación segmentadas, podemos llamar directamente a la biblioteca TensorFlow Model Analysis.

In [ ]:
import tensorflow_model_analysis as tfma

# Get the TFMA output result path and load the result.
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

# Show data sliced along feature column trip_start_hour.
tfma.view.render_slicing_metrics(
    tfma_result, slicing_column='trip_start_hour')

Esta visualización muestra las mismas métricas, pero calculadas en cada valor de característica de `trip_start_hour` en lugar de en todo el conjunto de evaluación.

TensorFlow Model Analysis admite muchas otras visualizaciones, como indicadores de equidad y el trazado de una serie temporal del rendimiento del modelo. Para obtener más información, consulte [el tutorial](https://www.tensorflow.org/tfx/tutorials/model_analysis/tfma_basic).

Dado que agregamos umbrales a nuestra configuración, el resultado de validación también está disponible. La presencia de un artefacto `blessing` indica que nuestro modelo pasó la validación. Dado que esta es la primera validación que se realiza, el candidato queda automáticamente aprobado.

In [ ]:
blessing_uri = evaluator.outputs['blessing'].get()[0].uri
!ls -l {blessing_uri}

Ahora también puede verificar el éxito al cargar el registro del resultado de la validación:

In [ ]:
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
print(tfma.load_validation_result(PATH_TO_RESULT))

### Pusher

El componente `Pusher` suele estar al final de una canalización de TFX. Comprueba si un modelo ha pasado la validación y, de ser así, exporta el modelo a `_serving_model_dir`.

In [ ]:
pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher, enable_cache=True)

Examinemos los artefactos de salida de `Pusher`.

In [ ]:
pusher.outputs

En particular, Pusher exportará su modelo en el formato SavedModel, que se ve así:

In [ ]:
push_uri = pusher.outputs['pushed_model'].get()[0].uri
model = tf.saved_model.load(push_uri)

for item in model.signatures.items():
  pp.pprint(item)

¡Hemos terminado nuestro recorrido por los componentes integrados en TFX!